# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = "/content/drive/MyDrive/23_2학기/빅처응/teamproject"
os.listdir(path)

['중간보고서_샘플_양식 (3).pptx',
 'links.csv',
 'w10',
 'tags.csv',
 'trash',
 'movies_refined_crawling.csv',
 'ratings_refined_timestamp.csv',
 'movies_refined.csv',
 'ratings_refined.csv',
 'movies_final.csv',
 'movie_user_imputed_knn1.csv',
 'movie_user_imputed_knn5.csv',
 'movie_user_imputed_knn3.csv',
 'item_based_life_pinda_backup.ipynb',
 'preprocessed',
 'Untitled0.ipynb',
 'item_based_life_pinda_final.ipynb']

In [3]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [4]:
ratings = pd.read_csv(path + "/ratings_refined_timestamp.csv", usecols=['userId', 'movieId', 'rating'])
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [5]:
movies = pd.read_csv(path + "/movies_final.csv")
movies

,movieId,title,genres,year
0,7,Sabrina1,Comedy Romance,1995
1,28,Persuasion1,Drama Romance,1995
2,73,"Misérables, Les1",Drama War,1995
3,88,Black Sheep1,Comedy,1996
4,95,Broken Arrow1,Action Adventure Thriller,1996
...,...,...,...,...
9732,193581,Black Butler: Book of the Atlantic,Action Animation Comedy Fantasy,2017
9733,193583,No Game No Life: Zero,Animation Comedy Fantasy,2017
9734,193585,Flint,Drama,2017
9735,193587,Bungo Stray Dogs: Dead Apple,Action Animation,2018


In [6]:
df = pd.merge(ratings, movies, on='movieId', how='left')
df

,userId,movieId,rating,title,genres,year
0,1,1,4.0,Toy Story,Adventure Animation Children Comedy Fantasy,1995
1,1,3,4.0,Grumpier Old Men,Comedy Romance,1995
2,1,6,4.0,Heat,Action Crime Thriller,1995
3,1,47,5.0,Seven (a.k.a. Se7en),Mystery Thriller,1995
4,1,50,5.0,"Usual Suspects, The",Crime Mystery Thriller,1995
...,...,...,...,...,...,...
100831,610,166534,4.0,Split,Drama Horror Thriller,2017
100832,610,168248,5.0,John Wick: Chapter Two,Action Crime Thriller,2017
100833,610,168250,5.0,Get Out,Horror,2017
100834,610,168252,5.0,Logan,Action Sci-Fi,2017


In [ ]:
df.columns[df.isna().any()].tolist()

[]

In [ ]:
df[df['title'].isnull()]

,userId,movieId,rating,title,genres,year


In [ ]:
df.head()
df.shape

(100836, 6)

In [7]:
# 테스트 데이터 미리 세팅
_, test_data = train_test_split(df[['userId', 'title', 'rating']],
                test_size=0.3, random_state=1234, stratify=df['userId'])
test_data

,userId,title,rating
13521,88,"Lord of the Rings: The Two Towers, The",5.0
53503,353,Babe,5.0
81738,517,White Christmas,4.5
56539,376,French Kiss,4.0
70981,453,Pleasantville,5.0
...,...,...,...
25130,177,Poltergeist,4.0
49184,318,Woman in the Dunes (Suna no onna),4.0
72396,466,Guardians of the Galaxy,3.5
11715,70,Shrek,4.0


# item-genre

In [37]:
# 모든 장르를 추출하여 중복 제거
all_genres = set(genre for genres in movies['genres'].str.split() for genre in genres)
all_genres = list(all_genres)

# 각 영화에 대해 장르에 속해 있는지 여부를 나타내는 이진 행렬 생성
binary_matrix = pd.DataFrame(0, index=movies['title'], columns=all_genres)

for index, row in movies.iterrows():
    title = row['title']
    genres = row['genres'].split()

    # 해당하는 장르에 1로 표시
    binary_matrix.loc[title, genres] = 1

binary_matrix

,Film-Noir,Romance,Documentary,Drama,War,Fantasy,Musical,Family,Sci-Fi,IMAX,...,Western,Biography,Adventure,Children,Thriller,History,Comedy,Animation,Short,Horror
title,,,,,,,,,,,,,,,,,,,,,
Sabrina1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Persuasion1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Misérables, Les1",0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Black Sheep1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Broken Arrow1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Black Butler: Book of the Atlantic,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
No Game No Life: Zero,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
Flint,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
movie_genre = binary_matrix.copy()

## 피어슨 상관계수

In [ ]:
# 피어슨 상관계수를 계산하여 유사도 행렬 생성
pearson_similarity = movie_genre.T.corr(method='pearson')

In [ ]:
# 결과 출력
pearson_similarity.head()

title,Sabrina1,Persuasion1,"Misérables, Les1",Black Sheep1,Broken Arrow1,Man of the Year1,Safe1,"Browning Version, The1","Hunted, The1",Little Women1,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
title,,,,,,,,,,,,,,,,,,,,,
Sabrina1,1.000000,0.454545,-0.090909,0.691564,-0.113961,-0.062869,-0.062869,-0.062869,-0.062869,-0.062869,...,0.269680,-0.090909,0.454545,-0.062869,-0.062869,0.269680,0.341882,-0.062869,-0.090909,0.691564
Persuasion1,0.454545,1.000000,0.454545,-0.062869,-0.113961,-0.062869,-0.062869,0.691564,-0.062869,0.691564,...,-0.134840,0.454545,0.454545,-0.062869,-0.062869,-0.134840,-0.113961,0.691564,-0.090909,-0.062869
"Misérables, Les1",-0.090909,0.454545,1.000000,-0.062869,-0.113961,-0.062869,-0.062869,0.691564,-0.062869,0.691564,...,-0.134840,0.454545,0.454545,-0.062869,-0.062869,-0.134840,-0.113961,0.691564,-0.090909,-0.062869
Black Sheep1,0.691564,-0.062869,-0.062869,1.000000,-0.078811,-0.043478,-0.043478,-0.043478,-0.043478,-0.043478,...,0.466252,-0.062869,0.691564,-0.043478,-0.043478,0.466252,0.551677,-0.043478,-0.062869,1.000000
Broken Arrow1,-0.113961,-0.113961,-0.113961,-0.078811,1.000000,-0.078811,0.551677,-0.078811,0.551677,-0.078811,...,0.169031,-0.113961,-0.113961,-0.078811,-0.078811,0.169031,-0.142857,-0.078811,0.341882,-0.078811


In [ ]:
# 높은수록 비슷한 영화
# 가장 높은 유저는 본인 자신 [0] index 제외
sample_movie = "'71"
pearson_similarity[sample_movie].sort_values(ascending=False)[1:30]

title
Lone Survivor                           1.000000
Army of Shadows (L'armée des ombres)    1.000000
Throne of Blood (Kumonosu jô)           1.000000
Green Zone                              1.000000
'71                                     1.000000
Hurt Locker, The                        1.000000
Dunkirk                                 1.000000
Rambo (Rambo 4)                         1.000000
Lord of War                             0.871780
Hell Ride                               0.845154
Zero Dark Thirty                        0.845154
Catch .44                               0.845154
Earthquake                              0.845154
Under Siege                             0.845154
Scorpio                                 0.845154
Airport 1975                            0.845154
Flightplan                              0.845154
Body of Lies                            0.845154
Painted Skin                            0.845154
In Hell                                 0.845154
Seeking Justic

## 코사인 유사도

In [ ]:
movie_similarity_matrix = cosine_similarity(movie_genre)
movie_similarity_matrix.shape

(9737, 9737)

In [ ]:
user_ids = movie_genre.index

movie_similarity = pd.DataFrame(movie_similarity_matrix,
                                index=user_ids, columns=user_ids)
print(movie_similarity.shape)
movie_similarity.head()

(9737, 9737)


title,Sabrina1,Persuasion1,"Misérables, Les1",Black Sheep1,Broken Arrow1,Man of the Year1,Safe1,"Browning Version, The1","Hunted, The1",Little Women1,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
title,,,,,,,,,,,,,,,,,,,,,
Sabrina1,1.000000,0.5,0.0,0.707107,0.0,0.0,0.00000,0.000000,0.00000,0.000000,...,0.353553,0.0,0.500000,0.0,0.0,0.353553,0.408248,0.000000,0.000000,0.707107
Persuasion1,0.500000,1.0,0.5,0.000000,0.0,0.0,0.00000,0.707107,0.00000,0.707107,...,0.000000,0.5,0.500000,0.0,0.0,0.000000,0.000000,0.707107,0.000000,0.000000
"Misérables, Les1",0.000000,0.5,1.0,0.000000,0.0,0.0,0.00000,0.707107,0.00000,0.707107,...,0.000000,0.5,0.500000,0.0,0.0,0.000000,0.000000,0.707107,0.000000,0.000000
Black Sheep1,0.707107,0.0,0.0,1.000000,0.0,0.0,0.00000,0.000000,0.00000,0.000000,...,0.500000,0.0,0.707107,0.0,0.0,0.500000,0.577350,0.000000,0.000000,1.000000
Broken Arrow1,0.000000,0.0,0.0,0.000000,1.0,0.0,0.57735,0.000000,0.57735,0.000000,...,0.288675,0.0,0.000000,0.0,0.0,0.288675,0.000000,0.000000,0.408248,0.000000


In [ ]:
# 높은수록 비슷한 영화
# 가장 높은 유저는 본인 자신 [0] index 제외
sample_movie = "'71"
movie_similarity[sample_movie].sort_values(ascending=False)[1:30]

title
Green Zone                                                  1.000000
Throne of Blood (Kumonosu jô)                               1.000000
Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka)    1.000000
Rambo (Rambo 4)                                             1.000000
Hurt Locker, The                                            1.000000
Dunkirk                                                     1.000000
Lone Survivor                                               1.000000
'71                                                         1.000000
Lord of War                                                 0.894427
Basic                                                       0.866025
Inglourious Basterds                                        0.866025
Memphis Belle                                               0.866025
Behind Enemy Lines II: Axis of Evil                         0.866025
Fail-Safe                                                   0.866025
Hazard                      

## 평점 예측

In [16]:
# 모델링 함수 선언
from tqdm.notebook import tqdm

def modeling(similarity_matrix, data):
    df_pred_all = pd.DataFrame()
    users = sorted(data['userId'].unique())
    all_titles = similarity_matrix.index
    n_titles = len(all_titles)

    for user in tqdm(users):
        idx = data[data['userId'] == user].index

        # 유사도
        watched_title = data.loc[idx, 'title'].tolist()
        sub_sim_mat = similarity_matrix.loc[watched_title]
        sub_sim_mat = sub_sim_mat.T.to_numpy()
        sim_N = np.sum(sub_sim_mat, axis=1) + 1

        # 평점 예측
        watched_title_y = data.loc[idx, 'rating']
        watched_title_y = np.array(watched_title_y.tolist()).reshape(-1, 1)

        pred_y = np.matmul(sub_sim_mat, watched_title_y).flatten() / sim_N

        user_list = [user] * n_titles
        cur_pred = pd.DataFrame(zip(user_list, all_titles, pred_y),
                                columns=['userId', 'title', 'pred_rating'])

        # 결과 기록
        df_pred_all = pd.concat([df_pred_all, cur_pred], axis=0)
    return df_pred_all


### 피어슨 결과

In [ ]:
df_pred_all_genre_pearson = modeling(pearson_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_genre_pearson.head()

,userId,title,pred_rating
0,1,Sabrina1,4.080404
1,1,Persuasion1,4.421699
2,1,"Misérables, Les1",4.549232
3,1,Black Sheep1,4.149004
4,1,Broken Arrow1,4.222394


In [ ]:
df_pred_all_genre_pearson['pred_rating'] = df_pred_all_genre_pearson['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_genre_pearson.describe()

,userId,pred_rating
count,5.939570e+06,5.939570e+06
mean,3.055000e+02,3.249429e+00
std,1.760916e+02,1.009828e+00
min,1.000000e+00,0.000000e+00
25%,1.530000e+02,2.859894e+00
50%,3.055000e+02,3.385622e+00
75%,4.580000e+02,3.819808e+00
max,6.100000e+02,5.000000e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_genre_pearson, on=['userId', 'title'], how='left')
result.head()

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,2.929808
1,353,Babe,5.0,3.623821
2,517,White Christmas,4.5,2.379732
3,376,French Kiss,4.0,3.701027
4,453,Pleasantville,5.0,3.983515


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.7458
MSE  : 0.94524
RMSE : 0.97224


### 코사인 결과

In [ ]:
df_pred_all_genre_cosine = modeling(movie_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_genre_cosine.head()

,userId,title,pred_rating
0,1,Sabrina1,4.206213
1,1,Persuasion1,4.374184
2,1,"Misérables, Les1",4.423022
3,1,Black Sheep1,4.209127
4,1,Broken Arrow1,4.270043


In [ ]:
df_pred_all_genre_cosine['pred_rating'] = df_pred_all_genre_cosine['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_genre_cosine.describe()

,userId,pred_rating
count,5.939570e+06,5.939570e+06
mean,3.055000e+02,3.229331e+00
std,1.760916e+02,8.096833e-01
min,1.000000e+00,0.000000e+00
25%,1.530000e+02,2.971619e+00
50%,3.055000e+02,3.388860e+00
75%,4.580000e+02,3.726632e+00
max,6.100000e+02,4.685262e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_genre_cosine, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,3.880201
1,353,Babe,5.0,3.532200
2,517,White Christmas,4.5,2.378456
3,376,French Kiss,4.0,3.826970
4,453,Pleasantville,5.0,3.966555
...,...,...,...,...
30246,177,Poltergeist,4.0,3.187030
30247,318,Woman in the Dunes (Suna no onna),4.0,3.826061
30248,466,Guardians of the Galaxy,3.5,3.735470
30249,70,Shrek,4.0,4.004555


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.70819
MSE  : 0.81511
RMSE : 0.90283


## tf-idf 처리 후

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# TF-IDF 변환
transformer = TfidfTransformer()
genre_tfidf = transformer.fit_transform(movie_genre.T).T

# TF-IDF 결과를 데이터프레임으로 변환
genre_tfidf_df = pd.DataFrame(genre_tfidf.toarray(), columns=movie_genre.columns, index=movie_genre.index)

# 결과 출력
genre_tfidf_df

,Horror,Action,Biography,Music,Crime,Thriller,Documentary,Short,Mystery,Drama,...,Family,Adventure,Children,Romance,Musical,War,Western,Comedy,Sci-Fi,Animation
title,,,,,,,,,,,,,,,,,,,,,
Sabrina1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.026552,0.0,0.000000,0.0,0.016454,0.0,0.000000
Persuasion1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.015328,...,0.0,0.000000,0.0,0.026552,0.0,0.000000,0.0,0.000000,0.0,0.000000
"Misérables, Les1",0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.015328,...,0.0,0.000000,0.0,0.000000,0.0,0.054905,0.0,0.000000,0.0,0.000000
Black Sheep1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.018592,0.0,0.000000
Broken Arrow1,0.0,0.023502,0.0,0.0,0.0,0.022757,0.0,0.0,0.0,0.000000,...,0.0,0.028956,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Black Butler: Book of the Atlantic,0.0,0.021651,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.013760,0.0,0.038127
No Game No Life: Zero,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.014937,0.0,0.041387
Flint,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.017320,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [ ]:
genre_tfidf_df.head(5)

,Horror,Action,Biography,Music,Crime,Thriller,Documentary,Short,Mystery,Drama,...,Family,Adventure,Children,Romance,Musical,War,Western,Comedy,Sci-Fi,Animation
title,,,,,,,,,,,,,,,,,,,,,
Sabrina1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.026552,0.0,0.000000,0.0,0.016454,0.0,0.0
Persuasion1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.015328,...,0.0,0.000000,0.0,0.026552,0.0,0.000000,0.0,0.000000,0.0,0.0
"Misérables, Les1",0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.015328,...,0.0,0.000000,0.0,0.000000,0.0,0.054905,0.0,0.000000,0.0,0.0
Black Sheep1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.018592,0.0,0.0
Broken Arrow1,0.0,0.023502,0.0,0.0,0.0,0.022757,0.0,0.0,0.0,0.000000,...,0.0,0.028956,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0


### 코사인

In [ ]:
cosine_sim = cosine_similarity(genre_tfidf_df, genre_tfidf_df)

In [ ]:
movie_similarity_tf = pd.DataFrame(cosine_sim, columns=movie_genre.T.columns, index=movie_genre.index)

In [ ]:
movie_similarity_tf.head()

title,Sabrina1,Persuasion1,"Misérables, Les1",Black Sheep1,Broken Arrow1,Man of the Year1,Safe1,"Browning Version, The1","Hunted, The1",Little Women1,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
title,,,,,,,,,,,,,,,,,,,,,
Sabrina1,1.000000,0.736161,0.000000,0.526749,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.132721,0.000000,0.385422,0.0,0.0,0.126555,0.136699,0.000000,0.000000,0.526749
Persuasion1,0.736161,1.000000,0.134434,0.000000,0.0,0.0,0.000000,0.499956,0.000000,0.499956,...,0.000000,0.159328,0.340783,0.0,0.0,0.000000,0.000000,0.499956,0.000000,0.000000
"Misérables, Les1",0.000000,0.134434,1.000000,0.000000,0.0,0.0,0.000000,0.268891,0.000000,0.268891,...,0.000000,0.085691,0.183283,0.0,0.0,0.000000,0.000000,0.268891,0.000000,0.000000
Black Sheep1,0.526749,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.251962,0.000000,0.731700,0.0,0.0,0.240258,0.259515,0.000000,0.000000,1.000000
Broken Arrow1,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.520896,0.000000,0.537952,0.000000,...,0.213268,0.000000,0.000000,0.0,0.0,0.203361,0.000000,0.000000,0.265643,0.000000


In [ ]:
df_pred_all_genre_tf_cosine = modeling(movie_similarity_tf, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_genre_tf_cosine.head()

,userId,title,pred_rating
0,1,Sabrina1,4.151156
1,1,Persuasion1,4.238935
2,1,"Misérables, Les1",4.351447
3,1,Black Sheep1,4.152613
4,1,Broken Arrow1,4.281360


In [ ]:
df_pred_all_genre_tf_cosine['pred_rating'] = df_pred_all_genre_tf_cosine['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_genre_tf_cosine.describe()

,userId,pred_rating
count,5.939570e+06,5.939570e+06
mean,3.055000e+02,3.125107e+00
std,1.760916e+02,8.189703e-01
min,1.000000e+00,0.000000e+00
25%,1.530000e+02,2.836750e+00
50%,3.055000e+02,3.286192e+00
75%,4.580000e+02,3.645174e+00
max,6.100000e+02,4.708531e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_genre_tf_cosine, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,3.951716
1,353,Babe,5.0,3.322972
2,517,White Christmas,4.5,2.537267
3,376,French Kiss,4.0,3.818610
4,453,Pleasantville,5.0,3.914861
...,...,...,...,...
30246,177,Poltergeist,4.0,3.157864
30247,318,Woman in the Dunes (Suna no onna),4.0,3.826613
30248,466,Guardians of the Galaxy,3.5,3.695366
30249,70,Shrek,4.0,3.999415


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.7096
MSE  : 0.81163
RMSE : 0.9009


### 피어슨

In [ ]:
# 피어슨 상관계수를 계산하여 유사도 행렬 생성
pearson_similarity = genre_tfidf_df.T.corr(method='pearson')

In [ ]:
pearson_similarity = pd.DataFrame(pearson_similarity, columns=genre_tfidf_df.T.columns, index=genre_tfidf_df.index)

In [ ]:
df_pred_all_genre_tf_pearson = modeling(pearson_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_genre_tf_pearson.head()

,userId,title,pred_rating
0,1,Sabrina1,3.756402
1,1,Persuasion1,3.614037
2,1,"Misérables, Les1",4.404876
3,1,Black Sheep1,4.011409
4,1,Broken Arrow1,4.241386


In [ ]:
df_pred_all_genre_tf_pearson['pred_rating'] = df_pred_all_genre_tf_pearson['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_genre_tf_pearson.describe()

,userId,pred_rating
count,5.939570e+06,5.939570e+06
mean,3.055000e+02,3.062148e+00
std,1.760916e+02,1.246188e+00
min,1.000000e+00,0.000000e+00
25%,1.530000e+02,2.553448e+00
50%,3.055000e+02,3.256259e+00
75%,4.580000e+02,3.789417e+00
max,6.100000e+02,5.000000e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_genre_tf_pearson, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,3.807359
1,353,Babe,5.0,2.651090
2,517,White Christmas,4.5,2.702606
3,376,French Kiss,4.0,3.663202
4,453,Pleasantville,5.0,3.904778
...,...,...,...,...
30246,177,Poltergeist,4.0,0.452092
30247,318,Woman in the Dunes (Suna no onna),4.0,3.867216
30248,466,Guardians of the Galaxy,3.5,3.589704
30249,70,Shrek,4.0,3.053260


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.79618
MSE  : 1.11368
RMSE : 1.05531


# Item-user

## 결측치 0으로 처리

In [8]:
movie_user = df.pivot_table(values='rating', index='title', columns='userId')
movie_user

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.5,NaN,NaN
xXx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,2.0
xXx: State of the Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5


In [9]:
movie_user_tmp = movie_user.copy().fillna(0)

### 코사인 유사도

In [10]:
movie_similarity_matrix = cosine_similarity(movie_user_tmp)
movie_similarity_matrix.shape

(9719, 9719)

In [11]:
# 영화-영화 유저관람 내역에 대한 유사도
movie_titles = movie_user_tmp.index

movie_similarity = pd.DataFrame(movie_similarity_matrix,
                                index=movie_titles, columns=movie_titles)
print(movie_similarity.shape)
movie_similarity.head()

(9719, 9719)


title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation,0.0,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight,0.0,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Salem's Lot,0.0,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Til There Was You,0.0,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [12]:
# 높은수록 비슷한 영화
# 가장 높은 영화는 본인 자신 [0] index 제외
sample_movie = "'71"
movie_similarity[sample_movie].sort_values(ascending=False)[1:30]

title
City of Lost Souls, The (Hyôryuu-gai)                          1.0
Clown                                                          1.0
Strange Circus (Kimyô na sâkasu)                               1.0
Ginger Snaps: Unleashed                                        1.0
Ginger Snaps Back: The Beginning                               1.0
Get on the Bus                                                 1.0
Collector, The2                                                1.0
Prince of Darkness                                             1.0
Gen-X Cops                                                     1.0
Stingray Sam                                                   1.0
Pulse (Kairo)                                                  1.0
Cooties                                                        1.0
Frontière(s)                                                   1.0
From Beyond                                                    1.0
Rapture-Palooza                                         

### 피어슨 상관계수

In [ ]:
# 피어슨 상관계수를 계산하여 유사도 행렬 생성
pearson_similarity = movie_user_tmp.T.corr(method='pearson')

In [ ]:
pearson_similarity.head()

title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,-0.001642,-0.002324,-0.001642,-0.002254,-0.001642,-0.006407,-0.001642,0.135943,-0.004325,...,-0.001642,0.339935,0.542247,0.706526,-0.001642,-0.007675,0.134327,0.325287,-0.008185,-0.001642
'Hellboy': The Seeds of Creation,-0.001642,1.000000,0.706526,-0.001642,-0.002254,-0.001642,-0.006407,-0.001642,-0.010568,-0.004325,...,-0.001642,-0.004589,-0.002808,-0.002324,-0.001642,-0.007675,-0.007744,-0.003594,-0.008185,-0.001642
'Round Midnight,-0.002324,0.706526,1.000000,-0.002324,-0.003191,-0.002324,0.170199,-0.002324,-0.014958,-0.006121,...,-0.002324,-0.006495,-0.003975,-0.003289,-0.002324,-0.010863,-0.010961,-0.005087,-0.011585,-0.002324
'Salem's Lot,-0.001642,-0.001642,-0.002324,1.000000,0.857269,-0.001642,-0.006407,-0.001642,-0.010568,-0.004325,...,-0.001642,-0.004589,-0.002808,-0.002324,-0.001642,-0.007675,-0.007744,-0.003594,-0.008185,-0.001642
'Til There Was You,-0.002254,-0.002254,-0.003191,0.857269,1.000000,-0.002254,-0.008797,-0.002254,-0.014510,-0.005938,...,-0.002254,-0.006301,-0.003856,-0.003191,-0.002254,-0.010538,-0.010632,-0.004935,-0.011238,-0.002254


In [ ]:
# 높은수록 비슷한 영화
# 가장 높은 영화는 본인 자신 [0] index 제외
sample_movie = "'71"
pearson_similarity[sample_movie].sort_values(ascending=False)[1:30]

title
V/H/S/2                                         1.0
New Police Story (Xin jing cha gu shi)          1.0
New York, New York                              1.0
Ninja                                           1.0
Ninja: Shadow of a Tear                         1.0
Oculus                                          1.0
One-Armed Swordsman, The (Dubei dao)            1.0
Only Lovers Left Alive                          1.0
Pact, The                                       1.0
Paranormal Activity 4                           1.0
Barking Dogs Never Bite (Flandersui gae)        1.0
Perfect Getaway, A                              1.0
Prince of Darkness                              1.0
Pulse (Kairo)                                   1.0
Rapture-Palooza                                 1.0
Reality                                         1.0
Red Hill                                        1.0
Requiem                                         1.0
Resolution                                      1.0
New On

### 평점예측

#### 피어슨 결과

In [ ]:
df_pred_all_user_pearson = modeling(pearson_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_pearson['pred_rating'] = df_pred_all_user_pearson['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_pearson.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,2.552921e+00
std,1.760916e+02,1.172776e+00
min,1.000000e+00,0.000000e+00
25%,1.530000e+02,1.988077e+00
50%,3.055000e+02,2.818935e+00
75%,4.580000e+02,3.365227e+00
max,6.100000e+02,5.000000e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_pearson, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,4.158245
1,353,Babe,5.0,3.414127
2,517,White Christmas,4.5,2.600252
3,376,French Kiss,4.0,3.631997
4,453,Pleasantville,5.0,3.922687
...,...,...,...,...
30246,177,Poltergeist,4.0,3.344931
30247,318,Woman in the Dunes (Suna no onna),4.0,3.855664
30248,466,Guardians of the Galaxy,3.5,3.845179
30249,70,Shrek,4.0,3.946565


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.68918
MSE  : 0.77199
RMSE : 0.87863


#### 코사인 결과

In [17]:
df_pred_all_user_cosine = modeling(movie_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [18]:
df_pred_all_user_cosine['pred_rating'] = df_pred_all_user_cosine['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_cosine.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,2.813380e+00
std,1.760916e+02,8.794980e-01
min,1.000000e+00,0.000000e+00
25%,1.530000e+02,2.381789e+00
50%,3.055000e+02,3.004586e+00
75%,4.580000e+02,3.442453e+00
max,6.100000e+02,4.592277e+00


In [19]:
result = pd.merge(test_data, df_pred_all_user_cosine, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,4.198600
1,353,Babe,5.0,3.492744
2,517,White Christmas,4.5,2.580006
3,376,French Kiss,4.0,3.843551
4,453,Pleasantville,5.0,3.953626
...,...,...,...,...
30246,177,Poltergeist,4.0,3.374389
30247,318,Woman in the Dunes (Suna no onna),4.0,3.844139
30248,466,Guardians of the Galaxy,3.5,3.851710
30249,70,Shrek,4.0,4.097160


In [20]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.69465
MSE  : 0.7918
RMSE : 0.88983


## knn - 1로 결측 처리

In [ ]:
from sklearn.impute import KNNImputer

# KNNImputer 객체 생성
imputer = KNNImputer(n_neighbors=1)

# pandas dataframe을 numpy array로 변환
matrix = movie_user.to_numpy()

# KNNImputer를 이용하여 결측치를 채움
matrix_imputed = imputer.fit_transform(matrix)

# numpy array를 다시 pandas dataframe으로 변환
movie_user_imputed = pd.DataFrame(matrix_imputed, index=movie_user.index, columns=movie_user.columns)
movie_user_imputed

In [ ]:
movie_user_imputed.to_csv(path + '/movie_user_imputed_knn1.csv')

k = 3, 5인 경우도 각각 똑같이 진행하여 파일로 불러왔습니다.

In [ ]:
movie_user_imputed = pd.read_csv(path + '/movie_user_imputed_knn1.csv', index_col='title')

### 코사인 유사도

In [ ]:
movie_similarity_matrix = cosine_similarity(movie_user_imputed)
movie_similarity_matrix.shape

(9719, 9719)

In [ ]:
# 영화-영화 유저관람 내역에 대한 유사도
movie_titles = movie_user_imputed.index

movie_similarity = pd.DataFrame(movie_similarity_matrix,
                                index=movie_titles, columns=movie_titles)
print(movie_similarity.shape)
movie_similarity.head()

(9719, 9719)


title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.952637,0.944666,0.949902,0.941266,0.948141,0.935332,0.943048,0.936891,0.945135,...,0.917250,0.943013,0.942201,0.938067,0.956244,0.940206,0.924449,0.913199,0.931891,0.942762
'Hellboy': The Seeds of Creation,0.952637,1.000000,0.953491,0.963340,0.960968,0.958342,0.926865,0.946248,0.943134,0.955478,...,0.920387,0.937559,0.947173,0.940694,0.960655,0.944705,0.930191,0.915940,0.933161,0.948382
'Round Midnight,0.944666,0.953491,1.000000,0.940593,0.943348,0.938067,0.932597,0.945249,0.943587,0.941952,...,0.917962,0.935856,0.948185,0.939179,0.955226,0.942959,0.932382,0.915842,0.937829,0.946309
'Salem's Lot,0.949902,0.963340,0.940593,1.000000,0.967269,0.958272,0.931754,0.947422,0.944325,0.957689,...,0.923890,0.944222,0.945272,0.939885,0.962406,0.944760,0.933485,0.921755,0.936217,0.946255
'Til There Was You,0.941266,0.960968,0.943348,0.967269,1.000000,0.951859,0.927833,0.943091,0.937086,0.951252,...,0.913822,0.939079,0.943320,0.934205,0.957358,0.940056,0.930774,0.918618,0.934411,0.947448


In [ ]:
# 높은수록 비슷한 영화
# 가장 높은 영화는 본인 자신 [0] index 제외
sample_movie = "'71"
movie_similarity[sample_movie].sort_values(ascending=False)[1:30]

title
Guest, The                                                                             1.0
Bird People in China, The (Chûgoku no chôjin)                                          1.0
Undisputed II: Last Man Standing                                                       1.0
Undisputed III: Redemption                                                             1.0
Invincible Shaolin                                                                     1.0
Sinister                                                                               1.0
Inside (À l'intérieur)                                                                 1.0
Perfect Getaway, A                                                                     1.0
House of the Devil, The                                                                1.0
From Beyond                                                                            1.0
Hatchet III                                                                         

### 피어슨 상관계수

In [ ]:
# 피어슨 상관계수를 계산하여 유사도 행렬 생성
pearson_similarity = movie_user_imputed.T.corr(method='pearson')

In [ ]:
pearson_similarity.head()

title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.289092,0.211108,0.267680,0.194310,0.296359,0.242550,0.253659,0.146499,0.192604,...,0.167119,0.248022,0.211601,0.232568,0.248126,0.149202,0.068448,0.123915,0.167596,0.184478
'Hellboy': The Seeds of Creation,0.289092,1.000000,0.282254,0.418160,0.424075,0.392693,0.085251,0.244559,0.171869,0.288073,...,0.157427,0.114120,0.223737,0.215752,0.248792,0.148253,0.079970,0.106020,0.127690,0.203799
'Round Midnight,0.211108,0.282254,1.000000,0.108126,0.203792,0.139196,0.194093,0.266146,0.218747,0.122602,...,0.160553,0.133521,0.276109,0.230377,0.202667,0.167434,0.149022,0.137176,0.224943,0.215223
'Salem's Lot,0.267680,0.418160,0.108126,1.000000,0.529440,0.406629,0.166952,0.279113,0.210103,0.342850,...,0.211814,0.229164,0.216412,0.222909,0.310262,0.172491,0.144648,0.187668,0.187414,0.193724
'Til There Was You,0.194310,0.424075,0.203792,0.529440,1.000000,0.355457,0.162695,0.263656,0.160410,0.293578,...,0.139225,0.206317,0.237217,0.194020,0.282528,0.159128,0.157142,0.188881,0.208026,0.261931


In [ ]:
# 높은수록 비슷한 영화
# 가장 높은 영화는 본인 자신 [0] index 제외
sample_movie = "'71"
pearson_similarity[sample_movie].sort_values(ascending=False)[1:30]

title
Sinister                                                                               1.0
Masked Avengers                                                                        1.0
Splinter                                                                               1.0
Lifted                                                                                 1.0
Stake Land                                                                             1.0
Last Winter, The                                                                       1.0
Strange Circus (Kimyô na sâkasu)                                                       1.0
Last Shift                                                                             1.0
Last Exorcism, The                                                                     1.0
Deadly Outlaw: Rekka (a.k.a. Violent Fire) (Jitsuroku Andô Noboru kyôdô-den: Rekka)    1.0
King of Comedy (Hei kek ji wong)                                                    

### 평점예측

#### 피어슨 결과

In [ ]:
df_pred_all_user_pearson = modeling(pearson_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_pearson['pred_rating'] = df_pred_all_user_pearson['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_pearson.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,3.328102e+00
std,1.760916e+02,4.796531e-01
min,1.000000e+00,7.767353e-01
25%,1.530000e+02,3.045152e+00
50%,3.055000e+02,3.367974e+00
75%,4.580000e+02,3.661006e+00
max,6.100000e+02,4.558281e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_pearson, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,3.974746
1,353,Babe,5.0,3.451836
2,517,White Christmas,4.5,2.428572
3,376,French Kiss,4.0,3.883865
4,453,Pleasantville,5.0,3.915751
...,...,...,...,...
30246,177,Poltergeist,4.0,3.383614
30247,318,Woman in the Dunes (Suna no onna),4.0,3.785703
30248,466,Guardians of the Galaxy,3.5,3.796602
30249,70,Shrek,4.0,4.026689


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.69627
MSE  : 0.78939
RMSE : 0.88848


#### 코사인 결과

In [ ]:
df_pred_all_user_cosine = modeling(movie_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_cosine['pred_rating'] = df_pred_all_user_cosine['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_cosine.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,3.589590e+00
std,1.760916e+02,4.687763e-01
min,1.000000e+00,1.206591e+00
25%,1.530000e+02,3.292419e+00
50%,3.055000e+02,3.642407e+00
75%,4.580000e+02,3.917688e+00
max,6.100000e+02,4.751932e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_cosine, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,3.968048
1,353,Babe,5.0,3.454562
2,517,White Christmas,4.5,2.382202
3,376,French Kiss,4.0,4.032545
4,453,Pleasantville,5.0,3.930571
...,...,...,...,...
30246,177,Poltergeist,4.0,3.374634
30247,318,Woman in the Dunes (Suna no onna),4.0,3.753541
30248,466,Guardians of the Galaxy,3.5,3.892336
30249,70,Shrek,4.0,4.249965


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.72991
MSE  : 0.87398
RMSE : 0.93487


## knn - 3로 결측 처리

In [ ]:
# knn-1과 같은 방식으로 csv 파일로 저장하여 그 과정은 생략하겠습니다.
movie_user_imputed = pd.read_csv(path + '/movie_user_imputed_knn3.csv', index_col='title')

### 코사인 유사도

In [ ]:
movie_similarity_matrix = cosine_similarity(movie_user_imputed)
movie_similarity_matrix.shape

(9719, 9719)

In [ ]:
# 영화-영화 유저관람 내역에 대한 유사도
movie_titles = movie_user_imputed.index

movie_similarity = pd.DataFrame(movie_similarity_matrix,
                                index=movie_titles, columns=movie_titles)
print(movie_similarity.shape)
movie_similarity.head()

(9719, 9719)


title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.984627,0.982616,0.981780,0.978462,0.978522,0.975872,0.979672,0.978333,0.980610,...,0.966789,0.979607,0.979380,0.977162,0.984455,0.980273,0.969105,0.970609,0.973765,0.979039
'Hellboy': The Seeds of Creation,0.984627,1.000000,0.985543,0.986450,0.984506,0.983518,0.974193,0.981737,0.978578,0.982210,...,0.969293,0.977634,0.977910,0.976565,0.985552,0.979880,0.971695,0.970937,0.972957,0.984442
'Round Midnight,0.982616,0.985543,1.000000,0.982163,0.980831,0.976844,0.977757,0.982856,0.977995,0.979873,...,0.970845,0.977826,0.977574,0.976664,0.983872,0.977475,0.972869,0.971292,0.975659,0.978721
'Salem's Lot,0.981780,0.986450,0.982163,1.000000,0.987527,0.982632,0.974673,0.980324,0.980178,0.982468,...,0.971345,0.978749,0.977910,0.977879,0.987704,0.978893,0.976009,0.972704,0.976530,0.981804
'Til There Was You,0.978462,0.984506,0.980831,0.987527,1.000000,0.977808,0.972735,0.982133,0.977135,0.981258,...,0.968751,0.979792,0.976824,0.976185,0.983397,0.978233,0.973126,0.972427,0.974332,0.980647


In [ ]:
# 높은수록 비슷한 영화
# 가장 높은 영화는 본인 자신 [0] index 제외
sample_movie = "'71"
movie_similarity[sample_movie].sort_values(ascending=False)[1:30]

title
Sacrament, The                                                                         1.0
Youth of the Beast (Yaju no seishun)                                                   1.0
Masked Avengers                                                                        1.0
House of the Devil, The                                                                1.0
Merantau                                                                               1.0
Hatchet III                                                                            1.0
Hatchet                                                                                1.0
Guest, The                                                                             1.0
Wrong Cops                                                                             1.0
Wrong                                                                                  1.0
From Beyond                                                                         

### 피어슨 상관계수

In [ ]:
# 피어슨 상관계수를 계산하여 유사도 행렬 생성
pearson_similarity = movie_user_imputed.T.corr(method='pearson')

In [ ]:
pearson_similarity.head()

title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.461410,0.435537,0.389953,0.343094,0.394011,0.407748,0.424851,0.382795,0.387862,...,0.328137,0.438316,0.463298,0.422895,0.456367,0.409540,0.260845,0.346481,0.326746,0.365982
'Hellboy': The Seeds of Creation,0.461410,1.000000,0.492163,0.506595,0.494414,0.509568,0.329843,0.451857,0.348820,0.394133,...,0.360388,0.343363,0.391021,0.374707,0.445928,0.353948,0.289310,0.323661,0.263218,0.497257
'Round Midnight,0.435537,0.492163,1.000000,0.401096,0.414140,0.344622,0.455314,0.515182,0.371676,0.362948,...,0.418309,0.387072,0.413639,0.408748,0.434117,0.323795,0.354010,0.361848,0.375738,0.354808
'Salem's Lot,0.389953,0.506595,0.401096,1.000000,0.609607,0.499190,0.363074,0.428776,0.420735,0.428660,...,0.421877,0.399116,0.410242,0.429537,0.553245,0.349067,0.422623,0.385397,0.386080,0.433500
'Til There Was You,0.343094,0.494414,0.414140,0.609607,1.000000,0.403233,0.354034,0.518419,0.379735,0.441529,...,0.390385,0.468306,0.419906,0.421291,0.458338,0.381993,0.385332,0.409991,0.369298,0.445372


In [ ]:
# 높은수록 비슷한 영화
# 가장 높은 영화는 본인 자신 [0] index 제외
sample_movie = "'71"
pearson_similarity[sample_movie].sort_values(ascending=False)[1:30]

title
Wrong Cops                                                                             1.0
From Beyond                                                                            1.0
Invincible Shaolin                                                                     1.0
Behind the Mask: The Rise of Leslie Vernon                                             1.0
Room for Romeo Brass, A                                                                1.0
Down Terrace                                                                           1.0
King of Comedy (Hei kek ji wong)                                                       1.0
Resolution                                                                             1.0
Reality                                                                                1.0
Bird People in China, The (Chûgoku no chôjin)                                          1.0
Last Exorcism, The                                                                  

### 평점예측

#### 피어슨 결과

In [ ]:
df_pred_all_user_pearson = modeling(pearson_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_pearson['pred_rating'] = df_pred_all_user_pearson['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_pearson.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,3.470833e+00
std,1.760916e+02,4.677837e-01
min,1.000000e+00,1.060696e+00
25%,1.530000e+02,3.195773e+00
50%,3.055000e+02,3.519927e+00
75%,4.580000e+02,3.784317e+00
max,6.100000e+02,4.602245e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_pearson, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,3.998846
1,353,Babe,5.0,3.448089
2,517,White Christmas,4.5,2.410475
3,376,French Kiss,4.0,3.969767
4,453,Pleasantville,5.0,3.915838
...,...,...,...,...
30246,177,Poltergeist,4.0,3.386446
30247,318,Woman in the Dunes (Suna no onna),4.0,3.778214
30248,466,Guardians of the Galaxy,3.5,3.863138
30249,70,Shrek,4.0,4.145752


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.70055
MSE  : 0.80323
RMSE : 0.89623


#### 코사인 결과

In [ ]:
df_pred_all_user_cosine = modeling(movie_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_cosine['pred_rating'] = df_pred_all_user_cosine['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_cosine.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,3.591204e+00
std,1.760916e+02,4.692347e-01
min,1.000000e+00,1.210922e+00
25%,1.530000e+02,3.294368e+00
50%,3.055000e+02,3.643472e+00
75%,4.580000e+02,3.918881e+00
max,6.100000e+02,4.758502e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_cosine, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,3.966327
1,353,Babe,5.0,3.452145
2,517,White Christmas,4.5,2.381116
3,376,French Kiss,4.0,4.033192
4,453,Pleasantville,5.0,3.929701
...,...,...,...,...
30246,177,Poltergeist,4.0,3.373150
30247,318,Woman in the Dunes (Suna no onna),4.0,3.752624
30248,466,Guardians of the Galaxy,3.5,3.894206
30249,70,Shrek,4.0,4.252345


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.73027
MSE  : 0.87472
RMSE : 0.93527


## knn - 5로 결측 처리

In [ ]:
# knn-1과 같은 방식으로 csv 파일로 저장하여 그 과정은 생략하겠습니다.
movie_user_imputed = pd.read_csv(path + '/movie_user_imputed_knn5.csv', index_col='title')

In [ ]:
movie_user_imputed

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,4.6,4.000000,2.3,4.4,4.2,3.0,4.1,4.0,4.0,3.5,...,4.4,4.2,3.8,4.2,3.5,4.0,4.0,3.3,3.40000,4.0
'Hellboy': The Seeds of Creation,3.6,4.200000,3.2,4.0,4.2,3.2,4.1,4.2,4.6,3.6,...,4.5,5.0,2.8,3.6,4.3,3.6,3.2,3.0,3.40000,4.6
'Round Midnight,4.6,4.200000,2.6,4.0,3.6,4.0,4.0,4.0,4.2,3.5,...,4.5,3.6,3.2,3.8,2.8,4.0,3.6,3.8,3.00000,4.5
'Salem's Lot,4.2,3.948276,2.2,4.8,4.2,3.6,3.1,3.6,3.6,3.0,...,4.6,3.2,3.8,3.6,3.7,4.0,3.4,3.6,3.27027,3.5
'Til There Was You,3.8,3.948276,0.5,3.8,2.8,3.6,4.1,3.2,3.8,3.3,...,4.6,3.6,4.0,3.4,3.1,4.0,3.8,2.8,3.80000,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,4.6,3.800000,0.5,4.2,4.4,4.0,3.8,3.6,3.0,2.6,...,4.1,2.6,5.0,3.6,3.3,3.8,3.8,4.5,3.40000,3.6
xXx,4.6,4.400000,1.3,2.8,3.8,3.4,2.3,3.2,1.0,3.6,...,4.4,3.2,3.2,3.8,2.9,3.7,3.8,3.5,3.00000,2.0
xXx: State of the Union,4.0,3.800000,1.3,3.0,3.6,3.2,2.0,3.4,3.2,4.1,...,4.3,3.4,3.0,3.0,2.9,2.8,2.6,2.6,3.20000,1.5


### 코사인 유사도

In [ ]:
movie_similarity_matrix = cosine_similarity(movie_user_imputed)
movie_similarity_matrix.shape

(9719, 9719)

In [ ]:
# 영화-영화 유저관람 내역에 대한 유사도
movie_titles = movie_user_imputed.index

movie_similarity = pd.DataFrame(movie_similarity_matrix,
                                index=movie_titles, columns=movie_titles)
print(movie_similarity.shape)
movie_similarity.head()

(9719, 9719)


title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.990244,0.989496,0.988860,0.986425,0.986493,0.982917,0.987720,0.985850,0.988630,...,0.981018,0.986039,0.985227,0.984746,0.989311,0.987438,0.980587,0.980073,0.984842,0.987258
'Hellboy': The Seeds of Creation,0.990244,1.000000,0.991891,0.990305,0.988193,0.989547,0.981605,0.987390,0.985663,0.987583,...,0.979634,0.985557,0.984052,0.983304,0.989544,0.986598,0.979482,0.978555,0.982981,0.988830
'Round Midnight,0.989496,0.991891,1.000000,0.989850,0.988677,0.987482,0.983095,0.989426,0.984517,0.987347,...,0.981941,0.986102,0.985382,0.984371,0.988415,0.986491,0.982613,0.981576,0.983411,0.988180
'Salem's Lot,0.988860,0.990305,0.989850,1.000000,0.992032,0.989483,0.982586,0.989523,0.986678,0.988132,...,0.981704,0.986736,0.985715,0.985425,0.990758,0.987476,0.984051,0.981407,0.984754,0.988392
'Til There Was You,0.986425,0.988193,0.988677,0.992032,1.000000,0.986451,0.981796,0.989984,0.983755,0.987896,...,0.981279,0.986755,0.985168,0.985114,0.989558,0.985869,0.982922,0.982476,0.982747,0.987347


In [ ]:
# 높은수록 비슷한 영화
# 가장 높은 영화는 본인 자신 [0] index 제외
sample_movie = "'71"
movie_similarity[sample_movie].sort_values(ascending=False)[1:30]

title
Room for Romeo Brass, A                          1.0
Sacrament, The                                   1.0
Behind the Mask: The Rise of Leslie Vernon       1.0
Wrong Cops                                       1.0
Merantau                                         1.0
Youth of the Beast (Yaju no seishun)             1.0
Guest, The                                       1.0
Masked Avengers                                  1.0
Invincible Shaolin                               1.0
Sinister                                         1.0
Hatchet                                          1.0
Hatchet III                                      1.0
Splinter                                         1.0
The Magnificent Ruffians                         1.0
Inside (À l'intérieur)                           1.0
Stake Land                                       1.0
Stingray Sam                                     1.0
Lifted                                           1.0
King of Comedy (Hei kek ji wong)        

### 피어슨 상관계수

In [ ]:
# 피어슨 상관계수를 계산하여 유사도 행렬 생성
pearson_similarity = movie_user_imputed.T.corr(method='pearson')

In [ ]:
pearson_similarity.head()

title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.574355,0.570029,0.546586,0.493773,0.486725,0.485581,0.524531,0.500333,0.564610,...,0.497424,0.506303,0.535564,0.518605,0.560864,0.526725,0.404564,0.454238,0.510996,0.515622
'Hellboy': The Seeds of Creation,0.574355,1.000000,0.649887,0.583702,0.540410,0.585315,0.424758,0.486935,0.472033,0.500822,...,0.442324,0.466972,0.479508,0.451986,0.546003,0.470380,0.345179,0.390752,0.427420,0.555877
'Round Midnight,0.570029,0.649887,1.000000,0.590143,0.581185,0.527747,0.493553,0.593678,0.455737,0.518570,...,0.526178,0.511502,0.542771,0.508541,0.527889,0.494230,0.472469,0.501188,0.465629,0.553949
'Salem's Lot,0.546586,0.583702,0.590143,1.000000,0.707554,0.605464,0.479055,0.599359,0.534834,0.550745,...,0.520328,0.536104,0.555153,0.544671,0.625513,0.533517,0.520010,0.497423,0.512200,0.564073
'Til There Was You,0.493773,0.540410,0.581185,0.707554,1.000000,0.528062,0.481440,0.645690,0.468048,0.575409,...,0.526189,0.565853,0.560461,0.558008,0.612997,0.511084,0.509589,0.546688,0.476821,0.559038


In [ ]:
# 높은수록 비슷한 영화
# 가장 높은 영화는 본인 자신 [0] index 제외
sample_movie = "'71"
pearson_similarity[sample_movie].sort_values(ascending=False)[1:30]

title
Room for Romeo Brass, A                          1.0
Invincible Shaolin                               1.0
Last Exorcism, The                               1.0
Stake Land                                       1.0
Last Shift                                       1.0
Last Winter, The                                 1.0
Bird People in China, The (Chûgoku no chôjin)    1.0
Inside (À l'intérieur)                           1.0
Sacrament, The                                   1.0
Wrong Cops                                       1.0
Wrong                                            1.0
Stingray Sam                                     1.0
Lifted                                           1.0
Hatchet                                          1.0
From Beyond                                      1.0
Down Terrace                                     1.0
Resolution                                       1.0
Reality                                          1.0
Masked Avengers                         

### 평점예측

#### 피어슨 결과

In [ ]:
df_pred_all_user_pearson = modeling(pearson_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_pearson['pred_rating'] = df_pred_all_user_pearson['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_pearson.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,3.503911e+00
std,1.760916e+02,4.681255e-01
min,1.000000e+00,1.087122e+00
25%,1.530000e+02,3.225120e+00
50%,3.055000e+02,3.556739e+00
75%,4.580000e+02,3.817773e+00
max,6.100000e+02,4.644559e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_pearson, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,3.976959
1,353,Babe,5.0,3.455446
2,517,White Christmas,4.5,2.400077
3,376,French Kiss,4.0,3.994076
4,453,Pleasantville,5.0,3.917735
...,...,...,...,...
30246,177,Poltergeist,4.0,3.382498
30247,318,Woman in the Dunes (Suna no onna),4.0,3.767781
30248,466,Guardians of the Galaxy,3.5,3.877435
30249,70,Shrek,4.0,4.174869


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.70398
MSE  : 0.81116
RMSE : 0.90065


#### 코사인 결과

In [ ]:
df_pred_all_user_cosine = modeling(movie_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_cosine['pred_rating'] = df_pred_all_user_cosine['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_cosine.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,3.591539e+00
std,1.760916e+02,4.693393e-01
min,1.000000e+00,1.211832e+00
25%,1.530000e+02,3.294811e+00
50%,3.055000e+02,3.643692e+00
75%,4.580000e+02,3.918869e+00
max,6.100000e+02,4.759571e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_cosine, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,3.965536
1,353,Babe,5.0,3.451677
2,517,White Christmas,4.5,2.380804
3,376,French Kiss,4.0,4.033474
4,453,Pleasantville,5.0,3.929563
...,...,...,...,...
30246,177,Poltergeist,4.0,3.372774
30247,318,Woman in the Dunes (Suna no onna),4.0,3.752284
30248,466,Guardians of the Galaxy,3.5,3.894520
30249,70,Shrek,4.0,4.252795


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.73038
MSE  : 0.87495
RMSE : 0.93539


## 정규화 이용하여 결측 처리

In [ ]:
movie_user_to_svd = movie_user.copy()
movie_user_to_svd

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.5,NaN,NaN
xXx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,2.0
xXx: State of the Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5


In [ ]:
from sklearn.decomposition import TruncatedSVD
import math

In [ ]:
# 결측을 컬럼 평균으로 채워주는 함수
def R_filled_in(rating_table):
    for col in range(len(rating_table.columns)):
        col_update=[]
        # 컬럼의 평균을 구한다.
        col_num = [i for i in rating_table.iloc[:,col] if math.isnan(i)==False]
        col_mean = sum(col_num)/len(col_num)

        # NaN을 가진 행은 위에서 구한 평균 값으로 채워준다.
        col_update = [i if math.isnan(i)==False else col_mean for i in rating_table.iloc[:,col]]

        # 리스트로 만든 업데이트된 한 컬럼을 기존에 데이터 프레임 컬럼에 새로 입혀준다.
        rating_table.iloc[:,col] = col_update

    return rating_table

In [ ]:
rating_R_filled = R_filled_in(movie_user_to_svd)
rating_R_filled

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,4.366379,3.948276,2.435897,3.555556,3.636364,3.493631,3.230263,3.574468,3.26087,3.278571,...,4.425743,3.392593,3.507953,3.48,3.210407,3.657399,3.786096,3.134176,3.27027,4.000000
'Hellboy': The Seeds of Creation,4.366379,3.948276,2.435897,3.555556,3.636364,3.493631,3.230263,3.574468,3.26087,3.278571,...,4.425743,3.392593,3.507953,3.48,3.210407,3.657399,3.786096,3.134176,3.27027,3.688556
'Round Midnight,4.366379,3.948276,2.435897,3.555556,3.636364,3.493631,3.230263,3.574468,3.26087,3.278571,...,4.425743,3.392593,3.507953,3.48,3.210407,3.657399,3.786096,3.134176,3.27027,3.688556
'Salem's Lot,4.366379,3.948276,2.435897,3.555556,3.636364,3.493631,3.230263,3.574468,3.26087,3.278571,...,4.425743,3.392593,3.507953,3.48,3.210407,3.657399,3.786096,3.134176,3.27027,3.688556
'Til There Was You,4.366379,3.948276,2.435897,3.555556,3.636364,3.493631,3.230263,3.574468,3.26087,3.278571,...,4.425743,3.392593,3.507953,3.48,3.210407,3.657399,3.786096,3.134176,3.27027,3.688556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,4.366379,3.948276,2.435897,3.555556,3.636364,3.493631,3.230263,3.574468,3.26087,3.278571,...,4.425743,3.392593,5.000000,3.48,3.210407,3.657399,3.786096,4.500000,3.27027,3.688556
xXx,4.366379,3.948276,2.435897,3.555556,3.636364,3.493631,3.230263,3.574468,1.00000,3.278571,...,4.425743,3.392593,3.507953,3.48,3.210407,3.657399,3.786096,3.500000,3.27027,2.000000
xXx: State of the Union,4.366379,3.948276,2.435897,3.555556,3.636364,3.493631,3.230263,3.574468,3.26087,3.278571,...,4.425743,3.392593,3.507953,3.48,3.210407,3.657399,3.786096,3.134176,3.27027,1.500000


In [ ]:
# 각 값에 행의 평균을 빼주어 정규화
def R_norm(rating_R_filled):
    for row in range(len(rating_R_filled)):
        # 각 값에 평균을 빼준 값들을 넣을 리스트 공간 생성.
        row_update=[]

        # 행 평균 값
        row_mean=sum(rating_R_filled.iloc[row,:])/len(rating_R_filled.iloc[row,:])

        # 해당 행의 모든 컬럼 값에 행 평균 값을 뺀다.
        row_update= [i-row_mean for i in rating_R_filled.iloc[row,:]]

        rating_R_filled.iloc[row,:] = row_update

    return rating_R_filled
R_norm(rating_R_filled)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,0.708649,0.290546,-1.221833,-0.102175,-0.021366,-0.164100,-0.427467,-0.083262,-0.396861,-0.379159,...,0.768012,-0.265138,-0.149777,-0.177730,-0.447323,-0.000331,0.128366,-0.523554,-0.387460,0.342270
'Hellboy': The Seeds of Creation,0.708503,0.290399,-1.221979,-0.102321,-0.021513,-0.164246,-0.427613,-0.083408,-0.397007,-0.379305,...,0.767866,-0.265284,-0.149923,-0.177877,-0.447469,-0.000477,0.128220,-0.523701,-0.387606,0.030680
'Round Midnight,0.709998,0.291895,-1.220484,-0.100826,-0.020017,-0.162751,-0.426118,-0.081913,-0.395512,-0.377810,...,0.769361,-0.263789,-0.148428,-0.176381,-0.445974,0.001018,0.129715,-0.522205,-0.386111,0.032175
'Salem's Lot,0.707362,0.289258,-1.223120,-0.103462,-0.022654,-0.165387,-0.428754,-0.084549,-0.398148,-0.380446,...,0.766725,-0.266425,-0.151064,-0.179018,-0.448610,-0.001618,0.127079,-0.524842,-0.388747,0.029539
'Til There Was You,0.708422,0.290318,-1.222060,-0.102402,-0.021594,-0.164327,-0.427694,-0.083489,-0.397088,-0.379386,...,0.767785,-0.265365,-0.150004,-0.177957,-0.447550,-0.000558,0.128139,-0.523782,-0.387687,0.030599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,0.700079,0.281976,-1.230403,-0.110745,-0.029936,-0.172670,-0.436037,-0.091832,-0.405431,-0.387729,...,0.759442,-0.273708,1.333700,-0.186300,-0.455893,-0.008901,0.119796,0.833700,-0.396030,0.022256
xXx,0.737723,0.319619,-1.192759,-0.073101,0.007707,-0.135026,-0.398394,-0.054189,-2.628657,-0.350085,...,0.797086,-0.236064,-0.120703,-0.148657,-0.418250,0.028742,0.157439,-0.128657,-0.358387,-1.628657
xXx: State of the Union,0.721176,0.303072,-1.209306,-0.089648,-0.008840,-0.151573,-0.414940,-0.070735,-0.384334,-0.366632,...,0.780539,-0.252611,-0.137250,-0.165204,-0.434796,0.012196,0.140893,-0.511028,-0.374933,-2.145204


In [ ]:
# 정규화 과정 진행
rating_R_norm = R_norm(rating_R_filled)

### 코사인 유사도

In [ ]:
movie_similarity_matrix = cosine_similarity(rating_R_norm)
movie_similarity_matrix.shape

(9719, 9719)

In [ ]:
# 영화-영화 유저관람 내역에 대한 유사도
movie_titles = rating_R_norm.index

movie_similarity = pd.DataFrame(movie_similarity_matrix,
                                index=movie_titles, columns=movie_titles)
print(movie_similarity.shape)
movie_similarity.head()

(9719, 9719)


title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.999085,0.999021,0.995430,0.993969,0.988961,0.939850,0.999094,0.878492,0.995759,...,0.993409,0.982391,0.995802,0.994873,0.997898,0.941561,0.906854,0.954205,0.920586,0.966235
'Hellboy': The Seeds of Creation,0.999085,1.000000,0.998513,0.995205,0.993745,0.988741,0.939644,0.998870,0.878657,0.995532,...,0.993188,0.981490,0.995993,0.996168,0.997674,0.941338,0.909994,0.958612,0.920384,0.966018
'Round Midnight,0.999021,0.998513,1.000000,0.995150,0.993683,0.988658,0.940676,0.998798,0.878655,0.995467,...,0.993110,0.981448,0.995924,0.996086,0.997599,0.941320,0.909779,0.958476,0.920229,0.965915
'Salem's Lot,0.995430,0.995205,0.995150,1.000000,0.998538,0.985130,0.936212,0.995220,0.875428,0.991904,...,0.989552,0.977884,0.992352,0.992538,0.994033,0.937895,0.906750,0.955151,0.917107,0.962532
'Til There Was You,0.993969,0.993745,0.993683,0.998538,1.000000,0.983669,0.934813,0.993754,0.874188,0.990451,...,0.988087,0.976464,0.990892,0.991069,0.992567,0.936557,0.905300,0.953694,0.915699,0.961095


### 평점예측

In [ ]:
df_pred_all_user_cosine_norm = modeling(movie_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_cosine_norm['pred_rating'] = df_pred_all_user_cosine_norm['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_cosine_norm.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,3.558691e+00
std,1.760916e+02,4.700221e-01
min,1.000000e+00,1.146752e+00
25%,1.530000e+02,3.270246e+00
50%,3.055000e+02,3.616492e+00
75%,4.580000e+02,3.880265e+00
max,6.100000e+02,4.748184e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_cosine_norm, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,3.921247
1,353,Babe,5.0,3.422798
2,517,White Christmas,4.5,2.384993
3,376,French Kiss,4.0,4.038038
4,453,Pleasantville,5.0,3.907829
...,...,...,...,...
30246,177,Poltergeist,4.0,3.369218
30247,318,Woman in the Dunes (Suna no onna),4.0,3.756875
30248,466,Guardians of the Galaxy,3.5,3.900275
30249,70,Shrek,4.0,4.205039


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.72477
MSE  : 0.85644
RMSE : 0.92544


### SVD n_components=60

In [ ]:
# 특이값 분해의 대각행렬 크기 (60,60)으로
svd = TruncatedSVD(n_components=60, n_iter=5)

In [ ]:
svd.fit(np.array(rating_R_norm))

TruncatedSVD(n_components=60)

In [ ]:
# 특이값 분해
U=svd.fit_transform(np.array(rating_R_norm))
Sigma=svd.explained_variance_ratio_
VT= svd.components_

In [ ]:
#행렬곱
ratings_reduced= pd.DataFrame(np.matmul(np.matmul(U, np.diag(Sigma)), VT), columns=movie_user.columns, index=movie_user.index)
ratings_reduced

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,0.003239,0.001310,-0.005690,-0.000538,-0.000158,-0.000790,-0.001921,-0.000436,-0.001846,-0.001847,...,0.003520,-0.001266,-0.000547,-0.000871,-0.002028,0.000153,0.000575,-0.002282,-0.001861,0.003516
'Hellboy': The Seeds of Creation,0.003277,0.001319,-0.005649,-0.000510,-0.000110,-0.000779,-0.001912,-0.000388,-0.001828,-0.001791,...,0.003528,-0.001216,-0.000575,-0.000832,-0.002050,0.000104,0.000575,-0.002428,-0.001795,0.000226
'Round Midnight,0.003265,0.001328,-0.005622,-0.000464,-0.000107,-0.000769,-0.001968,-0.000387,-0.001823,-0.001754,...,0.003527,-0.001224,-0.000673,-0.000826,-0.002056,0.000026,0.000589,-0.002301,-0.001784,0.000213
'Salem's Lot,0.003254,0.001349,-0.005586,-0.000436,-0.000095,-0.000744,-0.001966,-0.000384,-0.001811,-0.001719,...,0.003532,-0.001233,-0.000649,-0.000809,-0.002044,0.000020,0.000594,-0.002423,-0.001771,0.000093
'Til There Was You,0.003240,0.001343,-0.005612,-0.000410,-0.000087,-0.000713,-0.001886,-0.000378,-0.001822,-0.001747,...,0.003537,-0.001184,-0.000703,-0.000806,-0.002010,-0.000164,0.000606,-0.002377,-0.001778,0.000188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,0.003555,0.000794,-0.006899,-0.003433,-0.000361,-0.001214,-0.002284,-0.000428,-0.002411,-0.003932,...,0.003558,-0.001642,0.021130,-0.001133,-0.003594,0.003808,0.000942,0.015538,-0.002403,0.003090
xXx,0.001770,0.001994,-0.002890,-0.000038,0.001044,-0.000255,-0.003978,0.000955,-0.001410,-0.000207,...,0.003595,-0.000579,-0.002229,0.000543,-0.002305,-0.002107,0.001417,-0.001651,-0.000114,-0.029575
xXx: State of the Union,0.003476,0.001681,-0.004670,0.000312,0.000522,-0.000675,-0.002858,0.000088,-0.001433,-0.000626,...,0.003543,-0.000711,-0.001781,-0.000283,-0.002235,-0.001810,0.000314,-0.004891,-0.000972,-0.025910


In [ ]:
ratings_reduced

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,0.003239,0.001310,-0.005690,-0.000538,-0.000158,-0.000790,-0.001921,-0.000436,-0.001846,-0.001847,...,0.003520,-0.001266,-0.000547,-0.000871,-0.002028,0.000153,0.000575,-0.002282,-0.001861,0.003516
'Hellboy': The Seeds of Creation,0.003277,0.001319,-0.005649,-0.000510,-0.000110,-0.000779,-0.001912,-0.000388,-0.001828,-0.001791,...,0.003528,-0.001216,-0.000575,-0.000832,-0.002050,0.000104,0.000575,-0.002428,-0.001795,0.000226
'Round Midnight,0.003265,0.001328,-0.005622,-0.000464,-0.000107,-0.000769,-0.001968,-0.000387,-0.001823,-0.001754,...,0.003527,-0.001224,-0.000673,-0.000826,-0.002056,0.000026,0.000589,-0.002301,-0.001784,0.000213
'Salem's Lot,0.003254,0.001349,-0.005586,-0.000436,-0.000095,-0.000744,-0.001966,-0.000384,-0.001811,-0.001719,...,0.003532,-0.001233,-0.000649,-0.000809,-0.002044,0.000020,0.000594,-0.002423,-0.001771,0.000093
'Til There Was You,0.003240,0.001343,-0.005612,-0.000410,-0.000087,-0.000713,-0.001886,-0.000378,-0.001822,-0.001747,...,0.003537,-0.001184,-0.000703,-0.000806,-0.002010,-0.000164,0.000606,-0.002377,-0.001778,0.000188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,0.003555,0.000794,-0.006899,-0.003433,-0.000361,-0.001214,-0.002284,-0.000428,-0.002411,-0.003932,...,0.003558,-0.001642,0.021130,-0.001133,-0.003594,0.003808,0.000942,0.015538,-0.002403,0.003090
xXx,0.001770,0.001994,-0.002890,-0.000038,0.001044,-0.000255,-0.003978,0.000955,-0.001410,-0.000207,...,0.003595,-0.000579,-0.002229,0.000543,-0.002305,-0.002107,0.001417,-0.001651,-0.000114,-0.029575
xXx: State of the Union,0.003476,0.001681,-0.004670,0.000312,0.000522,-0.000675,-0.002858,0.000088,-0.001433,-0.000626,...,0.003543,-0.000711,-0.001781,-0.000283,-0.002235,-0.001810,0.000314,-0.004891,-0.000972,-0.025910


#### 코사인 유사도

In [ ]:
movie_similarity_matrix = cosine_similarity(ratings_reduced)
movie_similarity_matrix.shape

(9719, 9719)

In [ ]:
# 영화-영화 유저관람 내역에 대한 유사도
movie_titles = movie_user_to_svd.index

movie_similarity = pd.DataFrame(movie_similarity_matrix,
                                index=movie_titles, columns=movie_titles)
print(movie_similarity.shape)
movie_similarity.head()

(9719, 9719)


title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.997272,0.997088,0.996299,0.996494,0.984269,0.662304,0.986649,0.750862,0.973048,...,0.845705,0.938177,0.963229,0.931771,0.997061,0.771455,0.491354,0.745356,0.784237,0.995116
'Hellboy': The Seeds of Creation,0.997272,1.000000,0.999935,0.999775,0.999718,0.984705,0.679904,0.991154,0.736627,0.970239,...,0.862849,0.921534,0.976687,0.953371,0.999889,0.756144,0.538708,0.789082,0.800657,0.998115
'Round Midnight,0.997088,0.999935,1.000000,0.999872,0.999826,0.984174,0.683112,0.991639,0.734374,0.969571,...,0.864047,0.920606,0.977274,0.954263,0.999882,0.754117,0.541769,0.791097,0.803845,0.997988
'Salem's Lot,0.996299,0.999775,0.999872,1.000000,0.999878,0.983538,0.687907,0.992252,0.728106,0.967719,...,0.867604,0.916881,0.979171,0.957188,0.999698,0.748099,0.551011,0.797389,0.808545,0.997710
'Til There Was You,0.996494,0.999718,0.999826,0.999878,1.000000,0.983384,0.688950,0.992420,0.728473,0.967927,...,0.865829,0.918224,0.978340,0.955989,0.999648,0.748184,0.547686,0.795259,0.809392,0.997557


#### 평점 예측

In [ ]:
df_pred_all_user_svd_cosine = modeling(movie_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_svd_cosine['pred_rating'] = df_pred_all_user_svd_cosine['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_svd_cosine.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,3.433325e+00
std,1.760916e+02,6.220508e-01
min,1.000000e+00,0.000000e+00
25%,1.530000e+02,3.140261e+00
50%,3.055000e+02,3.505600e+00
75%,4.580000e+02,3.807241e+00
max,6.100000e+02,5.000000e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_svd_cosine, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,4.308355
1,353,Babe,5.0,3.904754
2,517,White Christmas,4.5,2.485505
3,376,French Kiss,4.0,3.829275
4,453,Pleasantville,5.0,4.017663
...,...,...,...,...
30246,177,Poltergeist,4.0,3.537115
30247,318,Woman in the Dunes (Suna no onna),4.0,3.771939
30248,466,Guardians of the Galaxy,3.5,3.903940
30249,70,Shrek,4.0,4.247009


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.66378
MSE  : 0.77415
RMSE : 0.87986


### SVD n_components=150

In [ ]:
# 특이값 분해의 대각행렬 크기 (150,150)으로
svd = TruncatedSVD(n_components=150, n_iter=5)

In [ ]:
svd.fit(np.array(rating_R_norm))

TruncatedSVD(n_components=150)

In [ ]:
# 특이값 분해
U=svd.fit_transform(np.array(rating_R_norm))
Sigma=svd.explained_variance_ratio_
VT= svd.components_

In [ ]:
# 시그마 행렬에 요소들에 루트
Sigma_sqrt=[]
for i in range(len(Sigma)):
    tmp=[]
    tmp=[math.sqrt(s) for s in np.diag(Sigma)[i]]
    Sigma_sqrt.append(tmp)

Sigma_sqrt=np.array(Sigma_sqrt)

In [ ]:
#행렬곱
ratings_reduced= pd.DataFrame(np.matmul(np.matmul(U, np.diag(Sigma)), VT), columns=movie_user.columns, index=movie_user.index)
ratings_reduced

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,0.003248,0.001308,-0.005705,-0.000533,-0.000157,-0.000809,-0.001920,-0.000432,-0.001847,-0.001859,...,0.003521,-0.001256,-0.000549,-0.000870,-0.002039,0.000147,0.000571,-0.002287,-0.001860,0.003553
'Hellboy': The Seeds of Creation,0.003311,0.001329,-0.005636,-0.000503,-0.000117,-0.000789,-0.001908,-0.000371,-0.001826,-0.001810,...,0.003527,-0.001221,-0.000579,-0.000836,-0.002074,0.000104,0.000549,-0.002418,-0.001787,0.000232
'Round Midnight,0.003277,0.001326,-0.005654,-0.000469,-0.000106,-0.000757,-0.001967,-0.000384,-0.001821,-0.001747,...,0.003522,-0.001221,-0.000669,-0.000837,-0.002105,0.000025,0.000583,-0.002307,-0.001782,0.000223
'Salem's Lot,0.003251,0.001352,-0.005555,-0.000426,-0.000094,-0.000761,-0.001986,-0.000392,-0.001813,-0.001710,...,0.003532,-0.001216,-0.000654,-0.000803,-0.002052,0.000016,0.000571,-0.002426,-0.001772,0.000087
'Til There Was You,0.003260,0.001343,-0.005548,-0.000391,-0.000083,-0.000760,-0.001934,-0.000393,-0.001806,-0.001678,...,0.003547,-0.001123,-0.000700,-0.000830,-0.002001,-0.000097,0.000605,-0.002390,-0.001773,0.000172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,0.004066,0.000774,-0.006896,-0.002416,-0.000514,-0.001236,-0.002031,-0.000327,-0.002322,-0.003661,...,0.003586,-0.001596,0.021365,-0.001261,-0.003455,0.003531,0.000672,0.015546,-0.002427,0.003301
xXx,0.001945,0.002051,-0.002823,-0.000026,0.000976,-0.000236,-0.003884,0.000739,-0.001953,0.000058,...,0.003657,-0.000230,-0.002341,0.000380,-0.001903,-0.002436,0.001450,-0.001463,-0.000127,-0.029344
xXx: State of the Union,0.003341,0.001701,-0.004451,0.000574,0.000567,-0.000480,-0.002636,-0.000050,-0.001574,-0.000692,...,0.003540,-0.000624,-0.001815,-0.000524,-0.002009,-0.001527,0.000838,-0.004853,-0.000967,-0.026266


In [ ]:
ratings_reduced

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,0.003248,0.001308,-0.005705,-0.000533,-0.000157,-0.000809,-0.001920,-0.000432,-0.001847,-0.001859,...,0.003521,-0.001256,-0.000549,-0.000870,-0.002039,0.000147,0.000571,-0.002287,-0.001860,0.003553
'Hellboy': The Seeds of Creation,0.003311,0.001329,-0.005636,-0.000503,-0.000117,-0.000789,-0.001908,-0.000371,-0.001826,-0.001810,...,0.003527,-0.001221,-0.000579,-0.000836,-0.002074,0.000104,0.000549,-0.002418,-0.001787,0.000232
'Round Midnight,0.003277,0.001326,-0.005654,-0.000469,-0.000106,-0.000757,-0.001967,-0.000384,-0.001821,-0.001747,...,0.003522,-0.001221,-0.000669,-0.000837,-0.002105,0.000025,0.000583,-0.002307,-0.001782,0.000223
'Salem's Lot,0.003251,0.001352,-0.005555,-0.000426,-0.000094,-0.000761,-0.001986,-0.000392,-0.001813,-0.001710,...,0.003532,-0.001216,-0.000654,-0.000803,-0.002052,0.000016,0.000571,-0.002426,-0.001772,0.000087
'Til There Was You,0.003260,0.001343,-0.005548,-0.000391,-0.000083,-0.000760,-0.001934,-0.000393,-0.001806,-0.001678,...,0.003547,-0.001123,-0.000700,-0.000830,-0.002001,-0.000097,0.000605,-0.002390,-0.001773,0.000172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,0.004066,0.000774,-0.006896,-0.002416,-0.000514,-0.001236,-0.002031,-0.000327,-0.002322,-0.003661,...,0.003586,-0.001596,0.021365,-0.001261,-0.003455,0.003531,0.000672,0.015546,-0.002427,0.003301
xXx,0.001945,0.002051,-0.002823,-0.000026,0.000976,-0.000236,-0.003884,0.000739,-0.001953,0.000058,...,0.003657,-0.000230,-0.002341,0.000380,-0.001903,-0.002436,0.001450,-0.001463,-0.000127,-0.029344
xXx: State of the Union,0.003341,0.001701,-0.004451,0.000574,0.000567,-0.000480,-0.002636,-0.000050,-0.001574,-0.000692,...,0.003540,-0.000624,-0.001815,-0.000524,-0.002009,-0.001527,0.000838,-0.004853,-0.000967,-0.026266


#### 코사인 유사도

In [ ]:
movie_similarity_matrix = cosine_similarity(ratings_reduced)
movie_similarity_matrix.shape

(9719, 9719)

In [ ]:
# 영화-영화 유저관람 내역에 대한 유사도
movie_titles = movie_user_to_svd.index

movie_similarity = pd.DataFrame(movie_similarity_matrix,
                                index=movie_titles, columns=movie_titles)
print(movie_similarity.shape)
movie_similarity.head()

(9719, 9719)


title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.997231,0.997069,0.996282,0.996069,0.982640,0.661332,0.986642,0.738382,0.972996,...,0.845681,0.937288,0.962979,0.931672,0.996949,0.768473,0.489503,0.742808,0.778415,0.992183
'Hellboy': The Seeds of Creation,0.997231,1.000000,0.999870,0.999732,0.999270,0.983033,0.678899,0.991114,0.724227,0.970155,...,0.862797,0.920583,0.976396,0.953271,0.999738,0.753106,0.536751,0.786521,0.795064,0.995172
'Round Midnight,0.997069,0.999870,1.000000,0.999843,0.999377,0.982521,0.682119,0.991622,0.722327,0.969518,...,0.864023,0.919665,0.977031,0.954186,0.999763,0.751140,0.539564,0.788545,0.798002,0.994982
'Salem's Lot,0.996282,0.999732,0.999843,1.000000,0.999463,0.981880,0.686908,0.992241,0.716088,0.967679,...,0.867579,0.915998,0.978930,0.957127,0.999593,0.745079,0.548926,0.794798,0.802677,0.994772
'Til There Was You,0.996069,0.999270,0.999377,0.999463,1.000000,0.981260,0.687469,0.991998,0.716032,0.967453,...,0.865463,0.917057,0.977751,0.955535,0.999130,0.744727,0.545254,0.792236,0.803123,0.994288


#### 평점 예측

In [ ]:
df_pred_all_user_svd_cosine = modeling(movie_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_svd_cosine['pred_rating'] = df_pred_all_user_svd_cosine['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_svd_cosine.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,3.432382e+00
std,1.760916e+02,6.225114e-01
min,1.000000e+00,0.000000e+00
25%,1.530000e+02,3.139122e+00
50%,3.055000e+02,3.504742e+00
75%,4.580000e+02,3.806380e+00
max,6.100000e+02,5.000000e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_svd_cosine, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,4.307544
1,353,Babe,5.0,3.907452
2,517,White Christmas,4.5,2.484880
3,376,French Kiss,4.0,3.828643
4,453,Pleasantville,5.0,4.020762
...,...,...,...,...
30246,177,Poltergeist,4.0,3.536680
30247,318,Woman in the Dunes (Suna no onna),4.0,3.772355
30248,466,Guardians of the Galaxy,3.5,3.902105
30249,70,Shrek,4.0,4.245370


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.66346
MSE  : 0.7734
RMSE : 0.87943


### SVD n_components=300

In [ ]:
# 특이값 분해의 대각행렬 크기 (300,300)으로
svd = TruncatedSVD(n_components=300, n_iter=5)

In [ ]:
svd.fit(np.array(rating_R_norm))

TruncatedSVD(n_components=300)

In [ ]:
# 특이값 분해
U=svd.fit_transform(np.array(rating_R_norm))
Sigma=svd.explained_variance_ratio_
VT= svd.components_

In [ ]:
#행렬곱
ratings_reduced= pd.DataFrame(np.matmul(np.matmul(U, np.diag(Sigma)), VT), columns=movie_user.columns, index=movie_user.index)
ratings_reduced

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,0.003253,0.001307,-0.005703,-0.000534,-0.000156,-0.000807,-0.001919,-0.000432,-0.001849,-0.001857,...,0.003522,-0.001255,-0.000549,-0.000870,-0.002039,0.000147,0.000569,-0.002288,-0.001860,0.003556
'Hellboy': The Seeds of Creation,0.003271,0.001336,-0.005659,-0.000494,-0.000102,-0.000781,-0.001911,-0.000384,-0.001835,-0.001798,...,0.003527,-0.001223,-0.000582,-0.000843,-0.002064,0.000099,0.000579,-0.002414,-0.001788,0.000233
'Round Midnight,0.003273,0.001323,-0.005633,-0.000472,-0.000100,-0.000768,-0.001964,-0.000381,-0.001816,-0.001744,...,0.003525,-0.001218,-0.000669,-0.000846,-0.002079,0.000028,0.000583,-0.002308,-0.001782,0.000222
'Salem's Lot,0.003240,0.001353,-0.005559,-0.000426,-0.000086,-0.000759,-0.001982,-0.000392,-0.001804,-0.001715,...,0.003531,-0.001223,-0.000653,-0.000808,-0.002059,0.000015,0.000562,-0.002427,-0.001770,0.000087
'Til There Was You,0.003238,0.001351,-0.005561,-0.000389,-0.000071,-0.000749,-0.001940,-0.000388,-0.001795,-0.001713,...,0.003545,-0.001146,-0.000701,-0.000823,-0.002021,-0.000096,0.000580,-0.002390,-0.001771,0.000171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,0.003851,0.000809,-0.006949,-0.002390,-0.000515,-0.001202,-0.002114,-0.000363,-0.002415,-0.003674,...,0.003609,-0.001635,0.021356,-0.001285,-0.003503,0.003502,0.000713,0.015540,-0.002421,0.003299
xXx,0.002101,0.002046,-0.002802,0.000016,0.001026,-0.000067,-0.003835,0.000751,-0.003283,-0.000004,...,0.003623,-0.000322,-0.002322,0.000374,-0.001920,-0.002465,0.001289,-0.001443,-0.000131,-0.029307
xXx: State of the Union,0.003408,0.001711,-0.004506,0.000565,0.000555,-0.000367,-0.002667,-0.000034,-0.001563,-0.000778,...,0.003519,-0.000665,-0.001799,-0.000451,-0.001994,-0.001562,0.000643,-0.004841,-0.000962,-0.026273


In [ ]:
ratings_reduced

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,0.003253,0.001307,-0.005703,-0.000534,-0.000156,-0.000807,-0.001919,-0.000432,-0.001849,-0.001857,...,0.003522,-0.001255,-0.000549,-0.000870,-0.002039,0.000147,0.000569,-0.002288,-0.001860,0.003556
'Hellboy': The Seeds of Creation,0.003271,0.001336,-0.005659,-0.000494,-0.000102,-0.000781,-0.001911,-0.000384,-0.001835,-0.001798,...,0.003527,-0.001223,-0.000582,-0.000843,-0.002064,0.000099,0.000579,-0.002414,-0.001788,0.000233
'Round Midnight,0.003273,0.001323,-0.005633,-0.000472,-0.000100,-0.000768,-0.001964,-0.000381,-0.001816,-0.001744,...,0.003525,-0.001218,-0.000669,-0.000846,-0.002079,0.000028,0.000583,-0.002308,-0.001782,0.000222
'Salem's Lot,0.003240,0.001353,-0.005559,-0.000426,-0.000086,-0.000759,-0.001982,-0.000392,-0.001804,-0.001715,...,0.003531,-0.001223,-0.000653,-0.000808,-0.002059,0.000015,0.000562,-0.002427,-0.001770,0.000087
'Til There Was You,0.003238,0.001351,-0.005561,-0.000389,-0.000071,-0.000749,-0.001940,-0.000388,-0.001795,-0.001713,...,0.003545,-0.001146,-0.000701,-0.000823,-0.002021,-0.000096,0.000580,-0.002390,-0.001771,0.000171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,0.003851,0.000809,-0.006949,-0.002390,-0.000515,-0.001202,-0.002114,-0.000363,-0.002415,-0.003674,...,0.003609,-0.001635,0.021356,-0.001285,-0.003503,0.003502,0.000713,0.015540,-0.002421,0.003299
xXx,0.002101,0.002046,-0.002802,0.000016,0.001026,-0.000067,-0.003835,0.000751,-0.003283,-0.000004,...,0.003623,-0.000322,-0.002322,0.000374,-0.001920,-0.002465,0.001289,-0.001443,-0.000131,-0.029307
xXx: State of the Union,0.003408,0.001711,-0.004506,0.000565,0.000555,-0.000367,-0.002667,-0.000034,-0.001563,-0.000778,...,0.003519,-0.000665,-0.001799,-0.000451,-0.001994,-0.001562,0.000643,-0.004841,-0.000962,-0.026273


#### 코사인 유사도

In [ ]:
movie_similarity_matrix = cosine_similarity(ratings_reduced)
movie_similarity_matrix.shape

(9719, 9719)

In [ ]:
# 영화-영화 유저관람 내역에 대한 유사도
movie_titles = movie_user_to_svd.index

movie_similarity = pd.DataFrame(movie_similarity_matrix,
                                index=movie_titles, columns=movie_titles)
print(movie_similarity.shape)
movie_similarity.head()

(9719, 9719)


title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu2,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.997207,0.997033,0.996276,0.996051,0.982614,0.661230,0.986641,0.737618,0.972992,...,0.845681,0.937095,0.962922,0.931670,0.996818,0.768161,0.489294,0.742595,0.778038,0.990526
'Hellboy': The Seeds of Creation,0.997207,1.000000,0.999799,0.999702,0.999228,0.982973,0.678778,0.991090,0.723449,0.970131,...,0.862776,0.920376,0.976316,0.953247,0.999592,0.752793,0.536488,0.786272,0.794615,0.993444
'Round Midnight,0.997033,0.999799,1.000000,0.999804,0.999328,0.982450,0.682032,0.991586,0.721566,0.969480,...,0.863989,0.919465,0.976938,0.954151,0.999593,0.750810,0.539297,0.788259,0.797583,0.993344
'Salem's Lot,0.996276,0.999702,0.999804,1.000000,0.999452,0.981845,0.686805,0.992235,0.715348,0.967671,...,0.867574,0.915795,0.978865,0.957120,0.999448,0.744780,0.548689,0.794573,0.802286,0.993086
'Til There Was You,0.996051,0.999228,0.999328,0.999452,1.000000,0.981210,0.687365,0.991980,0.715284,0.967435,...,0.865448,0.916840,0.977667,0.955517,0.998967,0.744438,0.545026,0.792004,0.802715,0.992604


#### 평점 예측

In [ ]:
df_pred_all_user_svd_cosine = modeling(movie_similarity, df)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
df_pred_all_user_svd_cosine['pred_rating'] = df_pred_all_user_svd_cosine['pred_rating'].apply(lambda x: min(5, max(0, x)))

# 결과 확인
df_pred_all_user_svd_cosine.describe()

,userId,pred_rating
count,5.928590e+06,5.928590e+06
mean,3.055000e+02,3.432273e+00
std,1.760916e+02,6.225873e-01
min,1.000000e+00,0.000000e+00
25%,1.530000e+02,3.138982e+00
50%,3.055000e+02,3.504647e+00
75%,4.580000e+02,3.806310e+00
max,6.100000e+02,5.000000e+00


In [ ]:
result = pd.merge(test_data, df_pred_all_user_svd_cosine, on=['userId', 'title'], how='left')
result

,userId,title,rating,pred_rating
0,88,"Lord of the Rings: The Two Towers, The",5.0,4.307615
1,353,Babe,5.0,3.907884
2,517,White Christmas,4.5,2.484797
3,376,French Kiss,4.0,3.828190
4,453,Pleasantville,5.0,4.020716
...,...,...,...,...
30246,177,Poltergeist,4.0,3.536675
30247,318,Woman in the Dunes (Suna no onna),4.0,3.772379
30248,466,Guardians of the Galaxy,3.5,3.901992
30249,70,Shrek,4.0,4.245199


In [ ]:
true_y = np.array(result['rating'])
pred_y = np.array(result['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 5))}")
print(f"MSE  : {str(round(mse, 5))}")
print(f"RMSE : {str(round(rmse, 5))}")

MAE  : 0.66344
MSE  : 0.77336
RMSE : 0.87941


# 추천 시스템

In [61]:
def get_recommendations_with_ranking(model_name, user_id):
    # 선호 장르 입력받기
    print("장르 종류 : Film-Noir, Romance, Documentary, Drama, War, Fantasy, Musical, Family, Sci-Fi, IMAX, Mystery, Action, Music, \n \t    Crime, Western, Biography, Adventure, Children, Thriller, History, Comedy, Animation, Short, Horror", end='\n\n')
    preferred_genre = input("선호하는 장르는 무엇인가요? (상관없으면 ENTER):")
    print('', end='\n\n')

    user_mov = df[df['userId'] == user_id]
    user_mov_pred = model_name[model_name['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')
    user_mov = pd.merge(user_mov, movies[['title', 'genres']], on='title', how='left')

    # 장르를 입력했다면 필터링
    if preferred_genre.strip():
      user_mov = user_mov[user_mov['genres_y'].str.contains(preferred_genre)]

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    # 랭킹 순으로 결과를 나타내기
    ranked_recommendations = movie_candidate[['title']].reset_index(drop=True)

    # 인덱스에 1씩 추가
    ranked_recommendations.index = ranked_recommendations.index + 1

    # title 컬럼명 변경
    ranked_recommendations = ranked_recommendations.rename(columns={'title': '좋아하실 만한 영화들이에요!'})


    # 결과를 가운데 정렬하는 HTML 스타일 적용
    styled_recommendations = (
        ranked_recommendations.style
        .set_properties(**{'text-align': 'center'})
        .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}])
    )

    return styled_recommendations

In [68]:
# 모델명, 확인하고픈 유저 id 입력 (아래는 예시)
get_recommendations_with_ranking(df_pred_all_user_cosine, 134)

장르 종류 : Film-Noir, Romance, Documentary, Drama, War, Fantasy, Musical, Family, Sci-Fi, IMAX, Mystery, Action, Music, 
 	    Crime, Western, Biography, Adventure, Children, Thriller, History, Comedy, Animation, Short, Horror

선호하는 장르는 무엇인가요? (상관없으면 ENTER):Music




,좋아하실 만한 영화들이에요!
1,"Nightmare Before Christmas, The"
2,Snow White and the Seven Dwarfs
3,"Wizard of Oz, The"
4,Willy Wonka & the Chocolate Factory
5,Pinocchio1
6,"Blues Brothers, The"
7,Mary Poppins
8,"South Park: Bigger, Longer and Uncut"
9,"Little Mermaid, The"
10,Shrek 2
